# Импорты

In [5]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import networkx as nx

# Класс сетевого анализа

In [6]:
class NetworkAnalysis:
    def __init__(self):
        self.activities = {}
        self.critical_path = []
        self.T_critical = 0
        self.variance_critical = 0
        
    def add_activity(self, name, dependencies, t_pes, t_ver, t_opt):
        self.activities[name] = {
            'dependencies': dependencies,
            't_pes': t_pes,
            't_ver': t_ver,
            't_opt': t_opt
        }
    
    def calculate_three_param_expected(self):
        """Расчет ожидаемого времени для трехпараметрической модели"""
        expected_times = {}
        for name, data in self.activities.items():
            t_pes = data['t_pes']
            t_ver = data['t_ver']
            t_opt = data['t_opt']
            t_expected = (t_pes + 4 * t_ver + t_opt) / 6
            expected_times[name] = round(t_expected)
        return expected_times
    
    def calculate_two_param_expected(self):
        """Расчет ожидаемого времени для двухпараметрической модели"""
        expected_times = {}
        for name, data in self.activities.items():
            t_pes = data['t_pes']
            t_opt = data['t_opt']
            t_expected = (3 * t_pes + 2 * t_opt) / 5
            expected_times[name] = round(t_expected)
        return expected_times
    
    def calculate_variance(self):
        """Расчет дисперсии продолжительности работ"""
        variances = {}
        for name, data in self.activities.items():
            t_pes = data['t_pes']
            t_opt = data['t_opt']
            variance = ((t_pes - t_opt) / 6) ** 2
            variances[name] = variance
        return variances
    
    def find_critical_path(self, expected_times):
        """Нахождение критического пути"""
        # Создаем граф
        G = nx.DiGraph()
        
        # Добавляем узлы и ребра
        for name, data in self.activities.items():
            duration = expected_times[name]
            G.add_node(name, duration=duration)
            
            if data['dependencies'] == ['-'] or data['dependencies'] == ['']:
                G.add_edge('start', name, weight=duration)
            else:
                for dep in data['dependencies']:
                    if dep != '-':
                        G.add_edge(dep, name, weight=duration)
        
        # Добавляем конечный узел
        for name in self.activities:
            has_dependents = False
            for other_name, other_data in self.activities.items():
                if name in other_data['dependencies']:
                    has_dependents = True
                    break
            if not has_dependents:
                G.add_edge(name, 'end', weight=0)
        
        # Находим самый длинный путь
        try:
            critical_path = nx.dag_longest_path(G, weight='weight')
            critical_length = nx.dag_longest_path_length(G, weight='weight')
            
            self.critical_path = [node for node in critical_path if node not in ['start', 'end']]
            self.T_critical = critical_length
            
            return self.critical_path, self.T_critical
            
        except Exception as e:
            print(f"Ошибка при поиске критического пути: {e}")
            return [], 0
    
    def calculate_critical_variance(self, variances):
        """Расчет дисперсии критического пути"""
        if not self.critical_path:
            return 0
            
        total_variance = 0
        for activity in self.critical_path:
            if activity in variances:
                total_variance += variances[activity]
        
        self.variance_critical = total_variance
        return total_variance
    
    def probability_completion_by_deadline(self, T_dir):
        """Вероятность выполнения проекта к заданному сроку"""
        if self.T_critical == 0 or self.variance_critical == 0:
            return 0
            
        sigma = np.sqrt(self.variance_critical)
        z = (T_dir - self.T_critical) / sigma
        
        probability = 0.5 + 0.5 * norm.cdf(z * np.sqrt(2))
        return probability
    
    def guaranteed_interval(self, P=0.9973):
        """Интервал гарантированного времени выполнения"""
        if self.variance_critical == 0:
            return (0, 0)
            
        sigma = np.sqrt(self.variance_critical)
        
        if P == 0.9973:
            # Правило трех сигм
            delta = 3 * sigma
        else:
            z = norm.ppf((1 + P) / 2)
            delta = z * sigma
            
        return (self.T_critical - delta, self.T_critical + delta)
    
    def max_completion_time_with_reliability(self, gamma=0.95):
        """Максимальный срок выполнения с заданной надежностью"""
        if self.variance_critical == 0:
            return self.T_critical
            
        sigma = np.sqrt(self.variance_critical)
        
        # Для функции Лапласа вида Φ(x) = ∫₀ˣ (1/√(2π))e^(-t²/2) dt
        z_gamma = norm.ppf(gamma)
        
        T_max = self.T_critical + z_gamma * sigma
        return T_max

# Основные расчеты

In [7]:
# Создаем экземпляр анализатора
analyzer = NetworkAnalysis()

# Данные из варианта 5 (примерные данные, нужно уточнить по вашему варианту)
activities_data = [
    # name, dependencies, t_pes, t_ver, t_opt
    ('b1', ['-'], 8, 5, 3),
    ('b2', ['-'], 10, 9, 4),
    ('b3', ['-'], 6, 2, 1),
    ('b4', ['b1'], 9, 7, 1),
    ('b5', ['b1'], 5, 4, 1),
    ('b6', ['b3'], 2, 1, 1),
    ('b7', ['b2', 'b5', 'b6'], 4, 2, 1),
    ('b8', ['b2', 'b5', 'b6'], 13, 5, 4),
    ('b9', ['b4', 'b7'], 8, 2, 1),
    ('b10', ['b3'], 17, 8, 6),
    ('b11', ['b2', 'b5', 'b6', 'b10'], 10, 8, 2)
]

# Добавляем активности
for activity in activities_data:
    analyzer.add_activity(*activity)

# Параметры задачи
T_dir = 25  # Директивный срок
gamma = 0.90  # Надежность
S = 12  # Стоимость одного дня

print("=" * 60)
print("АНАЛИЗ СЕТЕВОГО ГРАФИКА - ВАРИАНТ 5")
print("=" * 60)

# 1. Трехпараметрическая модель
print("\n1. ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ")
print("-" * 40)

# Расчет ожидаемых времен
three_param_times = analyzer.calculate_three_param_expected()
print("Ожидаемые продолжительности работ:")
for activity, time in three_param_times.items():
    print(f"  {activity}: {time} дней")

# Нахождение критического пути
critical_path_3p, T_critical_3p = analyzer.find_critical_path(three_param_times)
print(f"\nКритический путь: {' -> '.join(critical_path_3p)}")
print(f"Ожидаемое время выполнения проекта: {T_critical_3p} дней")

# Расчет дисперсий
variances = analyzer.calculate_variance()
print("\nДисперсии продолжительностей работ:")
for activity, var in variances.items():
    print(f"  {activity}: {var:.3f}")

# Дисперсия критического пути
variance_critical_3p = analyzer.calculate_critical_variance(variances)
sigma_3p = np.sqrt(variance_critical_3p)
print(f"\nДисперсия критического пути: {variance_critical_3p:.3f}")
print(f"Среднеквадратическое отклонение: {sigma_3p:.3f}")

# Вероятность выполнения к директивному сроку
prob_3p = analyzer.probability_completion_by_deadline(T_dir)
print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_3p:.3f} ({prob_3p*100:.1f}%)")

# Интервал гарантированного времени
guar_interval_3p = analyzer.guaranteed_interval()
print(f"Интервал гарантированного времени (P=0.9973): {guar_interval_3p[0]:.1f} - {guar_interval_3p[1]:.1f} дней")

# Максимальный срок с заданной надежностью
max_time_3p = analyzer.max_completion_time_with_reliability(gamma)
print(f"Максимальный срок с надежностью {gamma}: {max_time_3p:.1f} дней")

# 2. Двухпараметрическая модель
print("\n2. ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ")
print("-" * 40)

# Расчет ожидаемых времен
two_param_times = analyzer.calculate_two_param_expected()
print("Ожидаемые продолжительности работ:")
for activity, time in two_param_times.items():
    print(f"  {activity}: {time} дней")

# Нахождение критического пути
critical_path_2p, T_critical_2p = analyzer.find_critical_path(two_param_times)
print(f"\nКритический путь: {' -> '.join(critical_path_2p)}")
print(f"Ожидаемое время выполнения проекта: {T_critical_2p} дней")

# Дисперсия критического пути (та же, что и в трехпараметрической)
variance_critical_2p = analyzer.calculate_critical_variance(variances)
sigma_2p = np.sqrt(variance_critical_2p)
print(f"\nДисперсия критического пути: {variance_critical_2p:.3f}")
print(f"Среднеквадратическое отклонение: {sigma_2p:.3f}")

# Вероятность выполнения к директивному сроку
prob_2p = analyzer.probability_completion_by_deadline(T_dir)
print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_2p:.3f} ({prob_2p*100:.1f}%)")

# Интервал гарантированного времени
guar_interval_2p = analyzer.guaranteed_interval()
print(f"Интервал гарантированного времени (P=0.9973): {guar_interval_2p[0]:.1f} - {guar_interval_2p[1]:.1f} дней")

# Максимальный срок с заданной надежностью
max_time_2p = analyzer.max_completion_time_with_reliability(gamma)
print(f"Максимальный срок с надежностью {gamma}: {max_time_2p:.1f} дней")

# 3. Сравнение результатов
print("\n3. СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("-" * 40)

print(f"Разница в ожидаемом времени выполнения: {abs(T_critical_2p - T_critical_3p)} дней")
print(f"Разница в вероятности выполнения к сроку: {abs(prob_2p - prob_3p)*100:.1f}%")

# Анализ стоимости
print(f"\nАНАЛИЗ СТОИМОСТИ (S = {S} ден.ед./день):")
cost_3p = T_critical_3p * S
cost_2p = T_critical_2p * S
print(f"Ожидаемая стоимость (3-параметрическая): {cost_3p:.0f} ден.ед.")
print(f"Ожидаемая стоимость (2-параметрическая): {cost_2p:.0f} ден.ед.")

# Рекомендации
print("\n4. ВЫВОДЫ И РЕКОМЕНДАЦИИ")
print("-" * 40)

if prob_3p >= 0.7:
    print("Трехпараметрическая модель показывает хорошие шансы выполнения проекта в срок")
else:
    print("Трехпараметрическая модель показывает низкую вероятность выполнения в срок")
    
if prob_2p >= 0.7:
    print("Двухпараметрическая модель показывает хорошие шансы выполнения проекта в срок")
else:
    print("Двухпараметрическая модель показывает низкую вероятность выполнения в срок")

if T_critical_3p <= T_dir:
    print(f"Ожидаемое время выполнения ({T_critical_3p} дней) укладывается в директивный срок")
else:
    print(f"Ожидаемое время выполнения ({T_critical_3p} дней) превышает директивный срок")

# Критические работы для оптимизации
print(f"\nКритические работы для оптимизации: {', '.join(critical_path_3p)}")

АНАЛИЗ СЕТЕВОГО ГРАФИКА - ВАРИАНТ 5

1. ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ
----------------------------------------
Ожидаемые продолжительности работ:
  b1: 5 дней
  b2: 8 дней
  b3: 2 дней
  b4: 6 дней
  b5: 4 дней
  b6: 1 дней
  b7: 2 дней
  b8: 6 дней
  b9: 3 дней
  b10: 9 дней
  b11: 7 дней

Критический путь: b3 -> b10 -> b11
Ожидаемое время выполнения проекта: 18 дней

Дисперсии продолжительностей работ:
  b1: 0.694
  b2: 1.000
  b3: 0.694
  b4: 1.778
  b5: 0.444
  b6: 0.028
  b7: 0.250
  b8: 2.250
  b9: 1.361
  b10: 3.361
  b11: 1.778

Дисперсия критического пути: 5.833
Среднеквадратическое отклонение: 2.415

Вероятность выполнения к сроку 25 дней: 1.000 (100.0%)
Интервал гарантированного времени (P=0.9973): 10.8 - 25.2 дней
Максимальный срок с надежностью 0.9: 21.1 дней

2. ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ
----------------------------------------
Ожидаемые продолжительности работ:
  b1: 6 дней
  b2: 8 дней
  b3: 4 дней
  b4: 6 дней
  b5: 3 дней
  b6: 2 дней
  b7: 3 дней
  b8: 9 дней
  b9: 5 д